In [1]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

In [2]:
class Batch:
    def __init__(self, data, batchSize):
        self.data = np.copy(data)
        self.count = 0
        self.batchSize = batchSize
        np.random.shuffle(self.data)
    
    def getNextBatch(self):
        if self.count >= len(self.data) / self.batchSize:
            self.count = 0
            np.random.shuffle(self.data)
        start = self.count * self.batchSize
        end = start + self.batchSize
        toReturn = self.data[start:end]
        self.count += 1
        return toReturn

In [3]:
LEARNING_RATE_START = 0.005
LEARNING_RATE_END   = 0.0001
N_EPOCHS      = 300
anneal_rate = (-1.0 * np.log(LEARNING_RATE_END / LEARNING_RATE_START)) / float(N_EPOCHS)
batchSize = 50
trainingData = np.load("trainingData.npy")
testData = np.load("fullFinalData.npy")

In [4]:
np.random.shuffle(trainingData)
train_data = trainingData[:80000]
valid_data = trainingData[80000:]
trainBatch = Batch(train_data, batchSize)

In [12]:
sess = tf.Session()
beta = 0.01
A = tf.Variable(tf.random_normal([2048, 109], stddev=0.1))
B = tf.Variable(tf.random_normal([2048, 2048], stddev=0.1))
C = tf.Variable(tf.random_normal([1024, 2048], stddev=0.1)) 
D = tf.Variable(tf.random_normal([1024, 1024], stddev=0.1))
E = tf.Variable(tf.random_normal([1, 1024], stddev=0.1))
X = tf.placeholder(tf.float32, [None, 109])
y = tf.placeholder(tf.float32, [None, 1]) 
lr  = tf.placeholder(tf.float32)
regularizer = tf.nn.l2_loss(A) + tf.nn.l2_loss(B) + tf.nn.l2_loss(C) + tf.nn.l2_loss(D)
eval_dict={
    X   : valid_data[:,:-1],
    y   : valid_data[:,-1:]
}

In [13]:
hidden1 = tf.nn.relu(tf.matmul(A,tf.cast(tf.transpose(X), tf.float32)))
hidden2 = tf.nn.relu(tf.matmul(B, hidden1))
hidden3 = tf.nn.relu(tf.matmul(C, hidden2))
hidden4 = tf.nn.relu(tf.matmul(D, hidden3))
estimate = tf.nn.softmax(tf.matmul(E, hidden4))

In [14]:
loss = tf.losses.mean_squared_error(estimate, y) + beta * regularizer
# loss = tf.minimum(loss, 4.8)
# loss = tf.maximum(loss, -4.7)
metric = tf.metrics.mean_absolute_error(estimate, y)
optimizer = tf.train.AdamOptimizer(lr)
train_step = optimizer.minimize(loss)

In [15]:
saver = tf.train.Saver([A,B,C,D,E])

In [16]:
init = tf.global_variables_initializer()
sess.run(init)

In [21]:
#Restore Variables

saver = tf.train.import_meta_graph('ZestimateWeightV2.meta')
saver.restore(sess,tf.train.latest_checkpoint('./'))

INFO:tensorflow:Restoring parameters from ./ZestimateWeightV2


NotFoundError: Key Variable not found in checkpoint
	 [[Node: save/RestoreV2 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]

Caused by op u'save/RestoreV2', defined at:
  File "/anaconda/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/anaconda/lib/python2.7/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  F

In [20]:
epochSize = len(trainingData) / batchSize
best_loss = 0.066
curr_lr = LEARNING_RATE_START

for i in range(N_EPOCHS*epochSize):
    currBatch = trainBatch.getNextBatch()
    sess.run(train_step, feed_dict={
        X   : currBatch[:, :-1],
        y   : currBatch[:, -1:],
        lr  : curr_lr
    })
    checkpointed = False
    if i %  epochSize == 0:
        print("epoch #" + str(i /epochSize))
        tr_loss = sess.run(loss, eval_dict)
        if tr_loss < best_loss:
            best_loss = tr_loss
#             saver.save(sess, "ZestimateWeightV2")
            curr_lr = LEARNING_RATE_START * np.exp(-1.0 * anneal_rate * i)
            checkpointed = True
        print("checkPointed = " + str(checkpointed))
        print(tr_loss)

        

epoch #0
checkPointed = False
19.8965


KeyboardInterrupt: 

In [ ]:
preds = sess.run(estimate, { X: testData[:, 1:]})

In [270]:
np.save("NNPredictionsV2.npy", preds)

In [196]:
print preds.shape

(1, 2985217)